In [ ]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.1 MB/s eta 0:00:00


In [ ]:
# Load libraries
import pandas as pd
#import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PowerTransformer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import SMOTE
import missingno as msno

from catboost import CatBoostClassifier
from sklearn import preprocessing
import warnings
import re
import gc

warnings.filterwarnings('ignore')

In [ ]:
import google.colab.drive as gc
gc.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/MyDrive/python')

Mounted at /content/gdrive


In [ ]:
# Data set
df_customers = pd.read_csv("customers.csv", encoding='ISO-8859-1')
df_balances = pd.read_csv('balances.csv')
df_train = pd.read_csv('train.csv')
df_test= pd.read_csv('test.csv')

In [ ]:
print(df_balances.shape)
print(df_customers.shape)

(1367748, 19)
(151972, 8)


In [ ]:
# Uniendo el target del train con la tabla clientes
df_customers_01 = pd.merge(df_train, df_customers, on='ID', how='left')
df_customers_01.shape

(91183, 9)

In [ ]:
df_customers_01.isnull().sum()

ID                                0
TARGET                            0
CO_TIPO_SEXO                      0
EDAD                          15293
NO_DEPARTAMENTO               45473
NO_PROVINCIA                  45473
DE_CIIU                           0
PER_BANCARIZACION                 0
Dias_DesdeLa_bancarización        0
dtype: int64

In [ ]:
# Calcular el primer y tercer cuartil
Q1 = df_customers_01['EDAD'].quantile(0.25)
Q3 = df_customers_01['EDAD'].quantile(0.75)

# Calcular el rango intercuartílico (IQR)
IQR = Q3 - Q1

# Definir límites para detectar outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar los outliers
df_customers_01_filtered = df_customers_01[(df_customers_01['EDAD'] >= lower_bound) & (df_customers_01['EDAD'] <= upper_bound)]

In [ ]:
df_customers_01.isnull().sum()

ID                                0
TARGET                            0
CO_TIPO_SEXO                      0
EDAD                          15293
NO_DEPARTAMENTO               45473
NO_PROVINCIA                  45473
DE_CIIU                           0
PER_BANCARIZACION                 0
Dias_DesdeLa_bancarización        0
dtype: int64

In [ ]:
# Uniendo el target del train con la tabla clientes
df_customers_01_test = pd.merge(df_test, df_customers, on='ID', how='left')
df_customers_01_test

,ID,TARGET,CO_TIPO_SEXO,EDAD,NO_DEPARTAMENTO,NO_PROVINCIA,DE_CIIU,PER_BANCARIZACION,Dias_DesdeLa_bancarización
0,6f51cd85455148deaaa526a7e18bc6e3,NaN,1,45.0,NaN,NaN,ACTIV.DE ASESORAMIENTO EMPRESARIAL,200805,39569.0
1,1ebe738198e04d6db154578c929f2e36,NaN,2,47.0,NaN,NaN,OTRAS ACTIVIDADES NO CLASIFICAD.EN OTRA PARTE\r\n,201007,40360.0
2,44e170a4303e4ccdbdd0f25b65b89740,NaN,2,45.0,NaN,NaN,OTRAS ACTIVIDADES NO CLASIFICAD.EN OTRA PARTE\r\n,201111,40848.0
3,b9da0a6b004f4a23b9a5e88d3b20c613,NaN,1,36.0,AREQUIPA,AREQUIPA,OTRAS ACTIVIDADES NO CLASIFICAD.EN OTRA PARTE\r\n,201609,42614.0
4,9d7ec3782dbb48bfadb89dac733e8ca2,NaN,3,33.0,LAMBAYEQUE,CHICLAYO,PELUQUERIA Y OTROS,201510,42278.0
...,...,...,...,...,...,...,...,...,...
60784,c452ec2aa0d049a6bfa4e42d2bef085a,NaN,2,57.0,NaN,NaN,OTRAS ACTIVIDADES NO CLASIFICAD.EN OTRA PARTE\r\n,200809,39692.0
60785,dbe7683b4373495c8081a929091f1a95,NaN,3,NaN,NaN,NaN,OTRAS ACTIVIDADES NO CLASIFICAD.EN OTRA PARTE\r\n,201908,43678.0
60786,14205d6cf38f4c12ae4bc4db1843aaba,NaN,1,40.0,UCAYALI,CORONEL PORTILLO,OTRAS ACTIVID.DE TIPO SERVICIO NCP,201108,40756.0
60787,de668451e0344d0da9952366e93b487b,NaN,3,26.0,LA LIBERTAD,BOLIVAR,OTRAS ACTIV.RELAC. CON SALUD HUMANA,202106,44348.0


In [ ]:
df_customers_01_test.isnull().sum()

ID                                0
TARGET                        60789
CO_TIPO_SEXO                      0
EDAD                          10113
NO_DEPARTAMENTO               30271
NO_PROVINCIA                  30271
DE_CIIU                           0
PER_BANCARIZACION                 0
Dias_DesdeLa_bancarización        0
dtype: int64

In [ ]:
df = df_customers_01[['NO_DEPARTAMENTO']].isnull()+0
pd.concat([df,df_customers_01['Dias_DesdeLa_bancarización']], axis = 1).groupby('NO_DEPARTAMENTO').mean()

,Dias_DesdeLa_bancarización
NO_DEPARTAMENTO,
0,40723.530300
1,41328.441559


In [ ]:
# Cruzando balances con df_customers_01
df_customers_01 = pd.merge(df_balances, df_customers_01, on='ID', how='left')
df_customers_01

,PERIODO,ID,CANT_EMP_NEG,CANT_EMP_CONS,CANT_EMP_HIPOT,SALDO_MED_EMP,SALDO_PEQ_EMP,SALDO_MIC_EMP,SALDO_CONS_REV,SALDO_CONS_NO_REV,...,SALDO_DOLA_HIPOT,MAX_LINEA_DISP_U6M,TARGET,CO_TIPO_SEXO,EDAD,NO_DEPARTAMENTO,NO_PROVINCIA,DE_CIIU,PER_BANCARIZACION,Dias_DesdeLa_bancarización
0,202301,858130171a01415f93d40802fc9e192d,1,0,0,0.0,0.0,405.0000,0.0,0.0,...,0.0,1.1,0.0,2.0,64.0,UCAYALI,CORONEL PORTILLO,OTRAS ACTIVID.DE TIPO SERVICIO NCP,201001.0,40179.0
1,202302,858130171a01415f93d40802fc9e192d,1,0,0,0.0,0.0,346.5666,0.0,0.0,...,0.0,1.1,0.0,2.0,64.0,UCAYALI,CORONEL PORTILLO,OTRAS ACTIVID.DE TIPO SERVICIO NCP,201001.0,40179.0
2,202303,858130171a01415f93d40802fc9e192d,1,0,0,0.0,0.0,283.0869,0.0,0.0,...,0.0,1.1,0.0,2.0,64.0,UCAYALI,CORONEL PORTILLO,OTRAS ACTIVID.DE TIPO SERVICIO NCP,201001.0,40179.0
3,202304,858130171a01415f93d40802fc9e192d,1,0,0,0.0,0.0,217.9872,0.0,0.0,...,0.0,1.1,0.0,2.0,64.0,UCAYALI,CORONEL PORTILLO,OTRAS ACTIVID.DE TIPO SERVICIO NCP,201001.0,40179.0
4,202305,858130171a01415f93d40802fc9e192d,1,0,0,0.0,0.0,148.9428,0.0,0.0,...,0.0,1.1,0.0,2.0,64.0,UCAYALI,CORONEL PORTILLO,OTRAS ACTIVID.DE TIPO SERVICIO NCP,201001.0,40179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367743,202305,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,8417.4228,0.0,0.0,...,0.0,0.0,0.0,3.0,NaN,NaN,NaN,OTRAS ACTIVIDADES NO CLASIFICAD.EN OTRA PARTE\r\n,202108.0,44409.0
1367744,202306,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,8001.7551,0.0,0.0,...,0.0,0.0,0.0,3.0,NaN,NaN,NaN,OTRAS ACTIVIDADES NO CLASIFICAD.EN OTRA PARTE\r\n,202108.0,44409.0
1367745,202307,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,7540.8732,0.0,0.0,...,0.0,0.0,0.0,3.0,NaN,NaN,NaN,OTRAS ACTIVIDADES NO CLASIFICAD.EN OTRA PARTE\r\n,202108.0,44409.0
1367746,202308,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,7088.3262,0.0,0.0,...,0.0,0.0,0.0,3.0,NaN,NaN,NaN,OTRAS ACTIVIDADES NO CLASIFICAD.EN OTRA PARTE\r\n,202108.0,44409.0


In [ ]:
# Eliminando las columnas 'NO_DEPARTAMENTO', 'NO_DEPARTAMENTO', 'DE_CIIU','NO_PROVINCIA','PER_BANCARIZACION'
# Ya se ha creado previamente la columna
df_customers_01 = df_customers_01.drop(columns=['NO_DEPARTAMENTO', 'NO_DEPARTAMENTO', 'DE_CIIU','NO_PROVINCIA','PER_BANCARIZACION'])

In [ ]:
# Eliminando las filas que presentan NA en el target para quedarnos solo con la data de train
df_customers_01 = df_customers_01.dropna(subset=['TARGET'])
df_customers_01.shape

(820647, 23)

In [ ]:
df_customers_01[df_customers_01['ID'] == "093f542bfc5d48d8ab69612942d08538"]

,PERIODO,ID,CANT_EMP_NEG,CANT_EMP_CONS,CANT_EMP_HIPOT,SALDO_MED_EMP,SALDO_PEQ_EMP,SALDO_MIC_EMP,SALDO_CONS_REV,SALDO_CONS_NO_REV,...,SALDO_DOLA_NEG,CANT_EMP_DOL_CONS,SALDO_DOLA_CONS,CANT_EMP_DOL_HIPOT,SALDO_DOLA_HIPOT,MAX_LINEA_DISP_U6M,TARGET,CO_TIPO_SEXO,EDAD,Dias_DesdeLa_bancarización
1367739,202301,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,44409.0
1367740,202302,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,44409.0
1367741,202303,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,0.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,44409.0
1367742,202304,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,8843.7582,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,44409.0
1367743,202305,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,8417.4228,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,44409.0
1367744,202306,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,8001.7551,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,44409.0
1367745,202307,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,7540.8732,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,44409.0
1367746,202308,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,7088.3262,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,44409.0
1367747,202309,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,6630.0687,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,44409.0


In [ ]:
import pandas as pd
from google.colab import files

# Convertir el DataFrame a CSV
df_customers_01.to_csv('df_customers_01.csv', index=False)

# Descargar el archivo CSV a tu computadora local
files.download('df_customers_01.csv')

In [ ]:
df_customers_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 820647 entries, 0 to 1367747
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   PERIODO                     820647 non-null  int64  
 1   ID                          820647 non-null  object 
 2   CANT_EMP_NEG                820647 non-null  int64  
 3   CANT_EMP_CONS               820647 non-null  int64  
 4   CANT_EMP_HIPOT              820647 non-null  int64  
 5   SALDO_MED_EMP               820647 non-null  float64
 6   SALDO_PEQ_EMP               820647 non-null  float64
 7   SALDO_MIC_EMP               820647 non-null  float64
 8   SALDO_CONS_REV              820647 non-null  float64
 9   SALDO_CONS_NO_REV           820647 non-null  float64
 10  SALDO_HIPOT                 820647 non-null  float64
 11  SALDO_VENCIDO               820647 non-null  float64
 12  CANT_EMP_DOL_NEG            820647 non-null  float64
 13  SALDO_DOLA_NE

In [ ]:
Target_ID_train = df_customers_01.drop_duplicates(subset=["ID"],ignore_index=True)
Target_ID_train = Target_ID_train[['ID','TARGET','Dias_DesdeLa_bancarización']]
Target_ID_train

,ID,TARGET,Dias_DesdeLa_bancarización
0,858130171a01415f93d40802fc9e192d,0.0,40179.0
1,26b25bfb3c114bf89e4112eb21b1c553,0.0,38961.0
2,c43cec399839442f921c63e8a3fbb20e,1.0,38718.0
3,f91e50adadbf4fd986cb488ca81407c0,0.0,43009.0
4,11cd61cb672240429cf86a9781932118,0.0,38749.0
...,...,...,...
91178,f17e6f8b50d6459aa00a8dcddc622a30,0.0,43525.0
91179,95e8fe36ef164ba9b62ce31b3b2c320f,0.0,44440.0
91180,73702831906545009ab9e36c70062edd,0.0,44470.0
91181,811ab2a83be34f8e82cf5a132024b7a1,0.0,44562.0


In [ ]:
# Cambiando el tipo de variable
df_customers_01['CO_TIPO_SEXO'] = df_customers_01['CO_TIPO_SEXO'].astype('category')
df_customers_01['TARGET'] = df_customers_01['TARGET'].astype('category')
df_customers_01['Dias_DesdeLa_bancarización'] = df_customers_01['Dias_DesdeLa_bancarización'].astype(int)

In [ ]:
# Imputación y cambio del tipo de variable a edad
df_customers_01['EDAD'] = df_customers_01['EDAD'].fillna(df_customers_01['EDAD'].mean())
df_customers_01['EDAD'] = df_customers_01['EDAD'].astype(int)

In [ ]:
Variables_cuanti = ['CANT_EMP_NEG', 'CANT_EMP_CONS', 'CANT_EMP_HIPOT',
       'SALDO_MED_EMP', 'SALDO_PEQ_EMP', 'SALDO_MIC_EMP', 'SALDO_CONS_REV',
       'SALDO_CONS_NO_REV', 'SALDO_HIPOT', 'SALDO_VENCIDO', 'CANT_EMP_DOL_NEG',
       'SALDO_DOLA_NEG', 'CANT_EMP_DOL_CONS', 'SALDO_DOLA_CONS',
       'CANT_EMP_DOL_HIPOT', 'SALDO_DOLA_HIPOT', 'MAX_LINEA_DISP_U6M']

In [ ]:
df_customers_01

,PERIODO,ID,CANT_EMP_NEG,CANT_EMP_CONS,CANT_EMP_HIPOT,SALDO_MED_EMP,SALDO_PEQ_EMP,SALDO_MIC_EMP,SALDO_CONS_REV,SALDO_CONS_NO_REV,...,SALDO_DOLA_NEG,CANT_EMP_DOL_CONS,SALDO_DOLA_CONS,CANT_EMP_DOL_HIPOT,SALDO_DOLA_HIPOT,MAX_LINEA_DISP_U6M,TARGET,CO_TIPO_SEXO,EDAD,Dias_DesdeLa_bancarización
0,202301,858130171a01415f93d40802fc9e192d,1,0,0,0.0,0.0,405.0000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.1,0.0,2.0,64,40179
1,202302,858130171a01415f93d40802fc9e192d,1,0,0,0.0,0.0,346.5666,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.1,0.0,2.0,64,40179
2,202303,858130171a01415f93d40802fc9e192d,1,0,0,0.0,0.0,283.0869,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.1,0.0,2.0,64,40179
3,202304,858130171a01415f93d40802fc9e192d,1,0,0,0.0,0.0,217.9872,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.1,0.0,2.0,64,40179
4,202305,858130171a01415f93d40802fc9e192d,1,0,0,0.0,0.0,148.9428,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.1,0.0,2.0,64,40179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367743,202305,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,8417.4228,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46,44409
1367744,202306,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,8001.7551,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46,44409
1367745,202307,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,7540.8732,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46,44409
1367746,202308,093f542bfc5d48d8ab69612942d08538,1,0,0,0.0,0.0,7088.3262,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46,44409


### Variables nuevas

In [ ]:
aggregated_df = df_customers_01.groupby('ID').agg({
    'SALDO_MED_EMP': ['mean', 'std', 'max', 'min'],
    'SALDO_PEQ_EMP': ['mean', 'std', 'max', 'min'],
    'SALDO_MIC_EMP': ['mean', 'std', 'max', 'min'],
    'SALDO_CONS_REV': ['mean', 'std', 'max', 'min'],
    'SALDO_CONS_NO_REV': ['mean', 'std', 'max', 'min'],
    # Añade aquí más variables según sea necesario
})

# Aplanar el índice jerárquico y renombrar las columnas
aggregated_df.columns = ['_'.join(col).strip() for col in aggregated_df.columns.values]

# Resetear el índice para que 'ID' vuelva a ser una columna
aggregated_df.reset_index(inplace=True)

# Ejemplo de cómo calcular la tendencia de 'SALDO_MED_EMP' usando regresión lineal (esto es un pseudocódigo)
from sklearn.linear_model import LinearRegression

def calculate_trend(group):
    lr = LinearRegression()
    X = np.arange(len(group)).reshape(-1, 1)  # Convierte el índice del grupo en una matriz 2D
    y = group.values
    lr.fit(X, y)
    return lr.coef_[0]  # Retorna la pendiente de la tendencia

# Aplicar la función a cada grupo y añadir la nueva columna al DataFrame agregado
aggregated_df['SALDO_MED_EMP_trend'] = df_customers_01.groupby('ID')['SALDO_MED_EMP'].apply(calculate_trend)



In [ ]:
import google.colab.drive as gc
gc.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/MyDrive/python')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
aggregated_df.to_csv('/content/gdrive/MyDrive/python/aggregated_customers.csv', index=False)

In [ ]:
aggregated_df

,ID,SALDO_MED_EMP_mean,SALDO_MED_EMP_std,SALDO_MED_EMP_max,SALDO_MED_EMP_min,SALDO_PEQ_EMP_mean,SALDO_PEQ_EMP_std,SALDO_PEQ_EMP_max,SALDO_PEQ_EMP_min,SALDO_MIC_EMP_mean,...,SALDO_MIC_EMP_min,SALDO_CONS_REV_mean,SALDO_CONS_REV_std,SALDO_CONS_REV_max,SALDO_CONS_REV_min,SALDO_CONS_NO_REV_mean,SALDO_CONS_NO_REV_std,SALDO_CONS_NO_REV_max,SALDO_CONS_NO_REV_min,SALDO_MED_EMP_trend
0,00002e673be949b1afe485e71ff71048,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,0.0000,...,0.0000,0.000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,0.0000,NaN
1,00014b0427554bbbad91e0d8aeab3118,0.0,0.0,0.0,0.0,42683.458622,11485.996829,62865.3608,21246.4612,0.0000,...,0.0000,0.000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,0.0000,NaN
2,0001efa922ff405384514b5c6ed18294,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,0.0000,...,0.0000,0.000,0.000000,0.0000,0.0,4329.417411,647.778317,5176.5606,3351.6064,NaN
3,00024820367c454ba3822ec326ffd869,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,913.5747,...,0.0000,0.000,0.000000,0.0000,0.0,39.380733,79.597500,207.5000,0.0000,NaN
4,0002ccaee17946e895adb305ecaf288e,0.0,0.0,0.0,0.0,24677.719333,9633.132627,38925.9764,17734.2884,0.0000,...,0.0000,0.000,0.000000,0.0000,0.0,1261.417400,371.660542,1757.1100,683.9698,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91178,fffd817b33784b7b85fb95ee5a446f89,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,3985.7472,...,958.2381,0.000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,0.0000,NaN
91179,fffdb36006ff4b92bd4ca0ef4b74daa8,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,1365.3261,...,240.5295,0.000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,0.0000,NaN
91180,fffebeece7e3462c9ecd043d93bf8828,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,12296.1960,...,6389.0694,0.000,0.000000,0.0000,0.0,0.000000,0.000000,0.0000,0.0000,NaN
91181,ffff32ee7ea247bab950cfc94c7ba402,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0000,0.0000,0.0000,...,0.0000,143.507,123.267863,386.6954,0.0,6777.638900,1959.300981,8607.1000,3382.0757,NaN


In [ ]:
Data_train = pd.concat([Target_ID_train,df_customers_01], axis=1)
Data_train

,ID,TARGET,Dias_DesdeLa_bancarización,PERIODO,ID,CANT_EMP_NEG,CANT_EMP_CONS,CANT_EMP_HIPOT,SALDO_MED_EMP,SALDO_PEQ_EMP,...,SALDO_DOLA_NEG,CANT_EMP_DOL_CONS,SALDO_DOLA_CONS,CANT_EMP_DOL_HIPOT,SALDO_DOLA_HIPOT,MAX_LINEA_DISP_U6M,TARGET,CO_TIPO_SEXO,EDAD,Dias_DesdeLa_bancarización
0,858130171a01415f93d40802fc9e192d,0.0,40179.0,202301.0,858130171a01415f93d40802fc9e192d,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.1,0.0,2.0,64.0,40179.0
1,26b25bfb3c114bf89e4112eb21b1c553,0.0,38961.0,202302.0,858130171a01415f93d40802fc9e192d,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.1,0.0,2.0,64.0,40179.0
2,c43cec399839442f921c63e8a3fbb20e,1.0,38718.0,202303.0,858130171a01415f93d40802fc9e192d,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.1,0.0,2.0,64.0,40179.0
3,f91e50adadbf4fd986cb488ca81407c0,0.0,43009.0,202304.0,858130171a01415f93d40802fc9e192d,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.1,0.0,2.0,64.0,40179.0
4,11cd61cb672240429cf86a9781932118,0.0,38749.0,202305.0,858130171a01415f93d40802fc9e192d,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.1,0.0,2.0,64.0,40179.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367743,NaN,NaN,NaN,202305.0,093f542bfc5d48d8ab69612942d08538,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46.0,44409.0
1367744,NaN,NaN,NaN,202306.0,093f542bfc5d48d8ab69612942d08538,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46.0,44409.0
1367745,NaN,NaN,NaN,202307.0,093f542bfc5d48d8ab69612942d08538,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46.0,44409.0
1367746,NaN,NaN,NaN,202308.0,093f542bfc5d48d8ab69612942d08538,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46.0,44409.0


In [ ]:
duplicated_columns = Data_train.columns[Data_train.columns.duplicated()]
duplicated_columns

Index(['ID', 'TARGET', 'Dias_DesdeLa_bancarización'], dtype='object')

In [ ]:
Data_train = Data_train.loc[:, ~Data_train.columns.duplicated()] #elimina duplicados

In [ ]:
Data_train

,ID,TARGET,Dias_DesdeLa_bancarización,PERIODO,CANT_EMP_NEG,CANT_EMP_CONS,CANT_EMP_HIPOT,SALDO_MED_EMP,SALDO_PEQ_EMP,SALDO_MIC_EMP,...,SALDO_VENCIDO,CANT_EMP_DOL_NEG,SALDO_DOLA_NEG,CANT_EMP_DOL_CONS,SALDO_DOLA_CONS,CANT_EMP_DOL_HIPOT,SALDO_DOLA_HIPOT,MAX_LINEA_DISP_U6M,CO_TIPO_SEXO,EDAD
0,858130171a01415f93d40802fc9e192d,0.0,40179.0,202301.0,1.0,0.0,0.0,0.0,0.0,405.0000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1,2.0,64.0
1,26b25bfb3c114bf89e4112eb21b1c553,0.0,38961.0,202302.0,1.0,0.0,0.0,0.0,0.0,346.5666,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1,2.0,64.0
2,c43cec399839442f921c63e8a3fbb20e,1.0,38718.0,202303.0,1.0,0.0,0.0,0.0,0.0,283.0869,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1,2.0,64.0
3,f91e50adadbf4fd986cb488ca81407c0,0.0,43009.0,202304.0,1.0,0.0,0.0,0.0,0.0,217.9872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1,2.0,64.0
4,11cd61cb672240429cf86a9781932118,0.0,38749.0,202305.0,1.0,0.0,0.0,0.0,0.0,148.9428,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.1,2.0,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367743,NaN,NaN,NaN,202305.0,1.0,0.0,0.0,0.0,0.0,8417.4228,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46.0
1367744,NaN,NaN,NaN,202306.0,1.0,0.0,0.0,0.0,0.0,8001.7551,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46.0
1367745,NaN,NaN,NaN,202307.0,1.0,0.0,0.0,0.0,0.0,7540.8732,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46.0
1367746,NaN,NaN,NaN,202308.0,1.0,0.0,0.0,0.0,0.0,7088.3262,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,46.0


In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

def feature_engineering(dataframe):
    # Obtener todas las columnas numéricas
    numeric_cols = dataframe.select_dtypes(include=[np.number]).columns.tolist()

    # Eliminar 'ID' y 'TARGET' de la lista si existen
    if 'ID' in numeric_cols:
        numeric_cols.remove('ID')
    if 'TARGET' in numeric_cols:
        numeric_cols.remove('TARGET')

    # Crear interacciones entre todas las combinaciones de columnas numéricas
    for (col1, col2) in combinations(numeric_cols, 2):
        dataframe[f'{col1}_x_{col2}'] = dataframe[col1] * dataframe[col2]

    # Aplicar una transformación logarítmica a todas las columnas con solo valores positivos
    for col in numeric_cols:
        if (dataframe[col] > 0).all():
            dataframe[f'log_{col}'] = np.log(dataframe[col])

    return dataframe

# Asegúrate de que 'Data_train' es tu DataFrame
# Data_train = feature_engineering(Data_train)


In [ ]:
Data_train = feature_engineering(Data_train)

NameError: ignored

In [ ]:
Data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 857277 entries, 0 to 1367747
Data columns (total 23 columns):
 #   Column                      Non-Null Count   Dtype   
---  ------                      --------------   -----   
 0   ID                          91183 non-null   object  
 1   TARGET                      91183 non-null   float64 
 2   Dias_DesdeLa_bancarización  91183 non-null   float64 
 3   PERIODO                     820647 non-null  float64 
 4   CANT_EMP_NEG                820647 non-null  float64 
 5   CANT_EMP_CONS               820647 non-null  float64 
 6   CANT_EMP_HIPOT              820647 non-null  float64 
 7   SALDO_MED_EMP               820647 non-null  float64 
 8   SALDO_PEQ_EMP               820647 non-null  float64 
 9   SALDO_MIC_EMP               820647 non-null  float64 
 10  SALDO_CONS_REV              820647 non-null  float64 
 11  SALDO_CONS_NO_REV           820647 non-null  float64 
 12  SALDO_HIPOT                 820647 non-null  float64 
 13

In [ ]:
nan_columns = Data_train.columns[Data_train.isna().any()].tolist()
nan_columns

['ID',
 'TARGET',
 'Dias_DesdeLa_bancarización',
 'PERIODO',
 'CANT_EMP_NEG',
 'CANT_EMP_CONS',
 'CANT_EMP_HIPOT',
 'SALDO_MED_EMP',
 'SALDO_PEQ_EMP',
 'SALDO_MIC_EMP',
 'SALDO_CONS_REV',
 'SALDO_CONS_NO_REV',
 'SALDO_HIPOT',
 'SALDO_VENCIDO',
 'CANT_EMP_DOL_NEG',
 'SALDO_DOLA_NEG',
 'CANT_EMP_DOL_CONS',
 'SALDO_DOLA_CONS',
 'CANT_EMP_DOL_HIPOT',
 'SALDO_DOLA_HIPOT',
 'MAX_LINEA_DISP_U6M',
 'CO_TIPO_SEXO',
 'EDAD']

In [ ]:
Data_train = Data_train.drop(nan_columns, axis=1)

In [ ]:
Data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 857277 entries, 0 to 1367747
Empty DataFrame


In [ ]:
duplicated_columns = Data_train.columns[Data_train.columns.duplicated()]
duplicated_columns

Index(['ID', 'SALDO_MED_EMP_max', 'SALDO_PEQ_EMP_max', 'SALDO_MIC_EMP_max',
       'SALDO_CONS_REV_max', 'SALDO_CONS_NO_REV_max', 'SALDO_MED_EMP_min',
       'SALDO_PEQ_EMP_min', 'SALDO_MIC_EMP_min', 'SALDO_CONS_REV_min',
       'SALDO_CONS_NO_REV_min'],
      dtype='object')

In [ ]:
Data_train = Data_train.loc[:, ~Data_train.columns.duplicated()]

In [ ]:
df_customers_01_I = df_customers_01[Variables_cuanti]
import pandas as pd
import numpy as np

# Supongamos que tienes un DataFrame llamado 'df' con 17 columnas
# Reemplaza 'df' con el nombre real de tu DataFrame

# Calcula el número total de bloques
num_filas_por_bloque = 9
num_columnas = 17
num_bloques = len(df_customers_01_I) // num_filas_por_bloque

# Asegura que el número total de filas es múltiplo de num_filas_por_bloque
df_customers_01_I = df_customers_01_I.head(num_bloques * num_filas_por_bloque)

# Agrupa por bloques y calcula la media para cada bloque
# Medias
medias_por_bloque = df_customers_01_I.groupby(np.arange(len(df_customers_01_I)) // num_filas_por_bloque).mean()
df_medias = pd.DataFrame(medias_por_bloque)
df_medias.columns = [variable + '_media' for variable in df_medias.columns]
# Máximo
Maximo_por_bloque = df_customers_01_I.groupby(np.arange(len(df_customers_01_I)) // num_filas_por_bloque).max()
df_maximo = pd.DataFrame(Maximo_por_bloque)
df_maximo.columns = [variable + '_max' for variable in df_maximo.columns]
# Minimo
minimo_por_bloque = df_customers_01_I.groupby(np.arange(len(df_customers_01_I)) // num_filas_por_bloque).min()
df_minimo = pd.DataFrame(minimo_por_bloque)
df_minimo.columns = [variable + '_min' for variable in df_minimo.columns]
# Mediana
mediana_por_bloque = df_customers_01_I.groupby(np.arange(len(df_customers_01_I)) // num_filas_por_bloque).median()
df_mediana = pd.DataFrame(mediana_por_bloque)
df_mediana.columns = [variable + '_mediana' for variable in df_mediana.columns]

In [ ]:
## La moda solo se hallará para las variables de cantidad

In [ ]:
df_customers_01_moda = df_customers_01[['CANT_EMP_NEG', 'CANT_EMP_CONS', 'CANT_EMP_HIPOT','CANT_EMP_DOL_NEG','CANT_EMP_DOL_CONS','CANT_EMP_DOL_HIPOT']]
df_customers_01_moda = df_customers_01_moda.head(num_bloques * num_filas_por_bloque)

moda_por_bloque = df_customers_01_moda.groupby(np.arange(len(df_customers_01_I)) // num_filas_por_bloque).agg(lambda x: x.mode().iloc[0])
df_moda = pd.DataFrame(moda_por_bloque)
df_moda.columns = [variable + '_moda' for variable in df_moda.columns]

In [ ]:
Data_train = pd.concat([Target_ID_train, aggregated_df,df_medias, df_maximo,df_minimo,df_mediana,df_moda], axis=1)

In [ ]:
Data_train.head()

""
0
1
2
3
4


### Uniendo todos los resultados

In [ ]:
aggregated_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91183 entries, 0 to 91182
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID                      91183 non-null  object 
 1   SALDO_MED_EMP_mean      91183 non-null  float64
 2   SALDO_MED_EMP_std       91183 non-null  float64
 3   SALDO_MED_EMP_max       91183 non-null  float64
 4   SALDO_MED_EMP_min       91183 non-null  float64
 5   SALDO_PEQ_EMP_mean      91183 non-null  float64
 6   SALDO_PEQ_EMP_std       91183 non-null  float64
 7   SALDO_PEQ_EMP_max       91183 non-null  float64
 8   SALDO_PEQ_EMP_min       91183 non-null  float64
 9   SALDO_MIC_EMP_mean      91183 non-null  float64
 10  SALDO_MIC_EMP_std       91183 non-null  float64
 11  SALDO_MIC_EMP_max       91183 non-null  float64
 12  SALDO_MIC_EMP_min       91183 non-null  float64
 13  SALDO_CONS_REV_mean     91183 non-null  float64
 14  SALDO_CONS_REV_std      91183 non-null

In [ ]:
pd.set_option('display.max_seq_items', None)

# Ahora cuando imprimas las columnas, deberías ver todas sin truncar
print(Data_train.columns)

Index(['ID', 'TARGET', 'Dias_DesdeLa_bancarización', 'PERIODO', 'CANT_EMP_NEG',
       'CANT_EMP_CONS', 'CANT_EMP_HIPOT', 'SALDO_MED_EMP', 'SALDO_PEQ_EMP',
       'SALDO_MIC_EMP', 'SALDO_CONS_REV', 'SALDO_CONS_NO_REV', 'SALDO_HIPOT',
       'SALDO_VENCIDO', 'CANT_EMP_DOL_NEG', 'SALDO_DOLA_NEG',
       'CANT_EMP_DOL_CONS', 'SALDO_DOLA_CONS', 'CANT_EMP_DOL_HIPOT',
       'SALDO_DOLA_HIPOT', 'MAX_LINEA_DISP_U6M', 'CO_TIPO_SEXO', 'EDAD'],
      dtype='object')


In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

def feature_engineering(dataframe):
    # Obtener todas las columnas numéricas
    numeric_cols = dataframe.select_dtypes(include=[np.number]).columns.tolist()

    # Eliminar 'ID' y 'TARGET' de la lista si existen
    if 'ID' in numeric_cols:
        numeric_cols.remove('ID')
    if 'TARGET' in numeric_cols:
        numeric_cols.remove('TARGET')

    # Crear interacciones entre todas las combinaciones de columnas numéricas
    for (col1, col2) in combinations(numeric_cols, 2):
        dataframe[f'{col1}_x_{col2}'] = dataframe[col1] * dataframe[col2]

    # Aplicar una transformación logarítmica a todas las columnas con solo valores positivos
    for col in numeric_cols:
        if (dataframe[col] > 0).all():
            dataframe[f'log_{col}'] = np.log(dataframe[col])

    return dataframe

# Asegúrate de que 'Data_train' es tu DataFrame
# Data_train = feature_engineering(Data_train)



In [ ]:
Data_train = feature_engineering(Data_train)

In [ ]:
Data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54553 entries, 0 to 91182
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   ID                          54553 non-null  object  
 1   TARGET                      54553 non-null  float64 
 2   Dias_DesdeLa_bancarización  54553 non-null  float64 
 3   PERIODO                     54553 non-null  float64 
 4   CANT_EMP_NEG                54553 non-null  float64 
 5   CANT_EMP_CONS               54553 non-null  float64 
 6   CANT_EMP_HIPOT              54553 non-null  float64 
 7   SALDO_MED_EMP               54553 non-null  float64 
 8   SALDO_PEQ_EMP               54553 non-null  float64 
 9   SALDO_MIC_EMP               54553 non-null  float64 
 10  SALDO_CONS_REV              54553 non-null  float64 
 11  SALDO_CONS_NO_REV           54553 non-null  float64 
 12  SALDO_HIPOT                 54553 non-null  float64 
 13  SALDO_VENCIDO   

In [ ]:
Data_train = Data_train.dropna(subset=['PERIODO'])

In [ ]:
#Crección de las matriz X de variables predictoras y el Y el target
X,y = Data_train.drop(['ID','TARGET'], axis=1),Data_train[["TARGET"]]
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.2, random_state=1416,stratify=y)

In [ ]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43642 entries, 54738 to 29534
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   Dias_DesdeLa_bancarización  43642 non-null  float64 
 1   PERIODO                     43642 non-null  float64 
 2   CANT_EMP_NEG                43642 non-null  float64 
 3   CANT_EMP_CONS               43642 non-null  float64 
 4   CANT_EMP_HIPOT              43642 non-null  float64 
 5   SALDO_MED_EMP               43642 non-null  float64 
 6   SALDO_PEQ_EMP               43642 non-null  float64 
 7   SALDO_MIC_EMP               43642 non-null  float64 
 8   SALDO_CONS_REV              43642 non-null  float64 
 9   SALDO_CONS_NO_REV           43642 non-null  float64 
 10  SALDO_HIPOT                 43642 non-null  float64 
 11  SALDO_VENCIDO               43642 non-null  float64 
 12  CANT_EMP_DOL_NEG            43642 non-null  float64 
 13  SALDO_DOLA_N

In [ ]:
X_train = X_train.drop("SALDO_MED_EMP_trend", axis=1)

In [ ]:
y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91183 entries, 0 to 91182
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   TARGET  91183 non-null  float64
dtypes: float64(1)
memory usage: 1.4 MB


In [ ]:
X = X.dropna(subset=['TARGET'])

In [ ]:
### Entreno un modelo de Random Forest con 200 arboles y una semilla aleatoria 123
rf_clf = RandomForestClassifier(n_estimators=200,random_state =123)   ## forma del modelo

# Entreno el modelo con el .fit
rf_clf.fit(X_train.values,y_train.values.ravel())
## ordenando  las mejores variables de mayor a menor generando un bucle en este caso un for para
## extraer los nombres de las columnas y la ganancia(Gain) de las variables mas impactantes

features = []
for feature in zip(X.columns, rf_clf.feature_importances_):
    features.append(feature)

## genero un dataframe para visualizar mejor lo que hizo random forest con su selector multivariado
features_total = pd.DataFrame(features,columns=['Variables','Gain']).sort_values('Gain', ascending=False)
features_total

In [ ]:
y_pred = rf_clf.predict(X_test.values)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


0.8462102465401888
              precision    recall  f1-score   support

         0.0       0.86      0.98      0.92      9294
         1.0       0.36      0.05      0.09      1617

    accuracy                           0.85     10911
   macro avg       0.61      0.52      0.50     10911
weighted avg       0.78      0.85      0.79     10911



In [ ]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred, average='macro')
f1

0.5025248375411241

In [ ]:
importances = rf_clf.feature_importances_
importances

array([1.84217855e-01, 1.11472431e-01, 3.69230599e-02, 2.32746069e-02,
       2.06771314e-03, 2.94183710e-03, 9.70683108e-02, 1.41043387e-01,
       3.19670677e-02, 7.40003892e-02, 6.72784167e-03, 4.56590888e-02,
       1.56917009e-04, 2.14722780e-04, 1.21760362e-03, 4.39681230e-03,
       9.61906061e-05, 2.10465226e-04, 5.64661048e-02, 2.56891049e-02,
       1.54188490e-01])

In [ ]:
from sklearn.model_selection import GridSearchCV
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    # Add more parameters here
}
# Initialize the grid search model
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=5)
# Fit the grid search to the data
grid_search.fit(X_train.values, y_train.values.ravel())


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(n_estimators=200,
                                              random_state=123),
             param_grid={'max_depth': [None, 10, 20, 30],
                         'n_estimators': [100, 200, 300]})

In [ ]:
best_model = grid_search.best_estimator_


In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
f1 = f1_score(y_test, y_pred, average='macro')
f1

0.4771939837862405

### Punto de corte y selección de variables

In [ ]:
X = X.drop("SALDO_MED_EMP_trend", axis=1)

NameError: ignored

In [ ]:
## uso SelectFromModel poniendo lo parametros del modelo que se contruyo y un threshold que es un punto de corte que
## puede asumir de acuerdo a tu criterio con respecto a las variables que estén generando mas Gain
from sklearn.feature_selection import SelectFromModel
sfm = SelectFromModel(rf_clf, threshold=0.008)

# entrenamiento del selector
sfm.fit(X, y)

# contruyo una lista para quedarme con las mejores variables

variables = [] #### lista vacía
for feature_list_index in sfm.get_support(indices=True): ##
    variables.append(X.columns[feature_list_index])
    #variables

#features_total


In [ ]:
variables

In [ ]:
best_X_train = X_train[variables]
best_X_test  = X_test[variables]

In [ ]:
variables= list(set(variables))

In [ ]:
variables

## Construcción de modelos de Machine Learning

In [ ]:
## Usando la mejor selección de variables
best_X_train = X_train[variables]


best_X_test  = X_test[variables]


In [ ]:
rf_clf2 = RandomForestClassifier(n_estimators=200,random_state =123)   ## forma del modelo

# Entreno el modelo con el .fit
rf_clf2.fit(best_X_train.values,y_train.values.ravel())

In [ ]:
y_pred2 = rf_clf2.predict(best_X_test.values)

In [ ]:
f1 = f1_score(y_test, y_pred2, average='macro')
f1

In [ ]:
from sklearn.model_selection import GridSearchCV
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    # Add more parameters here
}
# Initialize the grid search model
grid_search2 = GridSearchCV(estimator=rf_clf2, param_grid=param_grid, cv=5)
# Fit the grid search to the data
grid_search2.fit(best_X_train.values, y_train.values.ravel())

In [ ]:
best_model2 = grid_search2.best_estimator_

In [ ]:
y_pred3 = best_model2.predict(best_X_test)

In [ ]:
f1 = f1_score(y_test, y_pred3, average='macro')
f1

In [ ]:
best_X_train.shape

In [ ]:
duplicated_columns = best_X_train.columns[best_X_train.columns.duplicated()]
duplicated_columns

In [ ]:
best_X_train = best_X_train.loc[:, ~best_X_train.columns.duplicated()]

In [ ]:
best_X_test = best_X_test.loc[:, ~best_X_test.columns.duplicated()]

In [ ]:
from time import time

# construimos un lightgbm, ojo que para el Y es el mismo porque es un vector univariado
best_model = LGBMClassifier()
best_model.fit(best_X_train, y_train)


# predecimos el train y test con la probabilidad para validar luego
predict_train_lg = best_model.predict_proba(best_X_train)[:,1]
predict_test_lg = best_model.predict_proba(best_X_test)[:,1]


# imprimimos el roc de train y test con la data real y la prediccion de la probabilidad del modelo
print("auc o Roc on training in LGBMClassifier data : {:.3f}".format(roc_auc_score(y_train, predict_train_lg)))
print("auc o Roc on testing in LGBMClassifier  data : {:.3f}".format(roc_auc_score(y_test, predict_test_lg)))

In [ ]:
predict_test_lg

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Supongamos que tienes las probabilidades predichas 'probs' y las etiquetas verdaderas 'y_true'
fpr, tpr, umbrales = roc_curve(y_test, predict_test_lg)

# Calcula el área bajo la curva ROC (ROC AUC)
roc_auc = roc_auc_score(y_test, predict_test_lg)

# Dibuja la curva ROC
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend()
plt.show()

# Encuentra el umbral que maximiza la suma de sensibilidad y especificidad
mejor_umbral = umbrales[np.argmax(tpr - fpr)]
print(f'Mejor Umbral: {mejor_umbral}')

Modelo Base

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
# Establece un nuevo punto de corte (umbral)
nuevo_punto_corte = 1390386861954071  # Por ejemplo, establece un nuevo umbral en 0.7

# Crea predicciones binarias basadas en el nuevo umbral
predicciones_binarias = (predict_test_lg > nuevo_punto_corte).astype(int)

# Calcula la matriz de confusión
matriz_confusion = confusion_matrix(y_test, predicciones_binarias)

# Imprime la matriz de confusión
print("Matriz de Confusión con nuevo punto de corte:")
print(matriz_confusion)
f_score = f1_score(y_test, predicciones_binarias,average = 'macro')
print(f_score)

In [ ]:
# Numero de arboles
n_estimators = [300, 400, 500]
# porcentaje de variables con la que se contruye un arbol
colsample_bytree = [0.7, 0.8, 0.9]
# profundidad del arbol
max_depth = [4, 6, 8]
# ratio de aprendizaje por cada arbol
learning_rate = [0.1, 0.15 ,0.2]
# creacion del grip search o grilla generando un diccionario
param_grid = {'n_estimators': n_estimators,
               'colsample_bytree': colsample_bytree,
               'max_depth': max_depth,
               'learning_rate': learning_rate}
print(param_grid)

In [ ]:
## Validación Cruzada
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
lgb = LGBMClassifier()
# Isntacia del grip search ponemos la grilla , la cantidad de kfolds para la validacion cruzada en este caso 5
# n_jobs -1 para la paralelizacion de la ejecucion en la optimizacion del modelo y verbose para mostrar de 2 en 2 los resultados
grid_search = GridSearchCV(estimator = lgb, param_grid = param_grid, cv = 5, n_jobs = -1, verbose = 2)
# FIT para ejecutarlo
X_best = X[variables]
 ## recordar que se usa toda la data completa y con las mejores variables
X_best = X_best.drop("SALDO_CONS_REV_min", axis=1)
grid_search.fit(X_best, y)
grid_search.best_params_ # mostramos los mejores hiperparametros

In [ ]:
from sklearn.model_selection import GridSearchCV


def hyperparameter_optimization(X_train, y_train):
    # Definir un grid de parámetros a buscar
    param_grid = {
        'num_leaves': [31, 50, 70],
        'reg_alpha': [0.1, 0.5, 1.0],
        'min_data_in_leaf': [20, 40, 60],
        'lambda_l1': [0, 0.1, 0.5],
        'lambda_l2': [0, 0.1, 0.5],
        # Añadir más parámetros aquí
    }

    # Inicializar el clasificador LightGBM
    lgbm = LGBMClassifier()

    # Configurar GridSearchCV
    grid_search = GridSearchCV(estimator=lgbm, param_grid=param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)

    # Realizar la búsqueda
    grid_search.fit(X_train, y_train)

    # Mostrar los mejores parámetros
    best_params = grid_search.best_params_
    print(f"Mejores parámetros: {best_params}")

    return grid_search.best_estimator_

In [ ]:
X_best = X[variables]
# Eliminar columnas duplicadas en el DataFrame
X_best =X_best.loc[:,~X_best.columns.duplicated()]

# Luego, continúa con la optimización de hiperparámetros
# best_model = hyperparameter_optimization(X_train, y_train)
hyperparameter_optimization(X_best, y)

In [ ]:
# Genero una funcion para validar la mejora de la curva ROC o auc para ver como aumenta el performance de mi modelo
def evaluate(model, test_features, test_labels):
    predictions = model.predict_proba(test_features)[:,1] ## recordar que para validar el roc se necesita el dato real y
    roc = roc_auc_score(test_labels,predictions)          ## probabilidad de la prediccion
    print('Model Performance')
    print('roc_auc_score = {:.12g}%.'.format(roc))
    return roc

In [ ]:
best_grid = grid_search.best_estimator_
grid_roc = evaluate(best_grid, best_X_test, y_test)
predictions = best_grid.predict_proba(best_X_test)[:,1]
predictions

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

# Supongamos que tienes las probabilidades predichas 'probs' y las etiquetas verdaderas 'y_true'
fpr, tpr, umbrales = roc_curve(y_test, predictions)

# Calcula el área bajo la curva ROC (ROC AUC)
roc_auc = roc_auc_score(y_test, predictions)

# Dibuja la curva ROC
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend()
plt.show()

# Encuentra el umbral que maximiza la suma de sensibilidad y especificidad
mejor_umbral = umbrales[np.argmax(tpr - fpr)]
print(f'Mejor Umbral: {mejor_umbral}')

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
# Establece un nuevo punto de corte (umbral)
nuevo_punto_corte = 0.13588207993239668  # Por ejemplo, establece un nuevo umbral en 0.7

# Crea predicciones binarias basadas en el nuevo umbral
predicciones_binarias = (predict_test_lg > nuevo_punto_corte).astype(int)

# Calcula la matriz de confusión
matriz_confusion = confusion_matrix(y_test, predicciones_binarias)

# Imprime la matriz de confusión
print("Matriz de Confusión con nuevo punto de corte:")
print(matriz_confusion)
f_score = f1_score(y_test, predicciones_binarias)
print(f_score)

In [ ]:
base_model = LGBMClassifier()
base_model.fit(best_X_train, y_train)
base_roc = evaluate(base_model, best_X_test, y_test)

## pasamos la funcion evaluate para sacar el resultado del modelo
best_grid = grid_search.best_estimator_
grid_roc = evaluate(best_grid, best_X_test, y_test)
print('Mejora en {:.12g}%.'.format( 100 * (grid_roc - base_roc) / base_roc)) ## le damos diseño para imprimir con .format
## podemos visualizar el aumento o las mejora en 3.5%